In [1]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D, Conv2D, MaxPooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.utils import to_categorical
from keras.optimizers import Adam

import pickle
from sklearn.model_selection import train_test_split 
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
import os
import csv
from PIL import Image


Using TensorFlow backend.


In [2]:
(train_set, train_label_raw), (X_test, y_test) = mnist.load_data()
train_label = to_categorical(train_label_raw)
train_data = np.asarray(train_set).reshape(60000, 28, 28, 1)

In [3]:
gen_images_raw = pd.read_csv('generated_images_47600_0.923000.csv', header=None)
X_fake_raw = np.asarray(gen_images_raw).reshape(100000, 28, 28, 1)

gen_labels_raw = pd.read_csv('generated_labels_47600_0.923000.csv', header=None)
y_fake_raw = to_categorical(gen_labels_raw)
X_fake, y_fake = shuffle(X_fake_raw, y_fake_raw)

In [4]:
class classifier():
    def __init__(self):
        
        self.model = Sequential()
        # model.add(Conv2D(5, kernel_size=3, activation='relu', input_shape=(28,28,1)))
        self.model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(28,28,1)))
        self.model.add(MaxPooling2D(pool_size=(4, 4)))
        self.model.add(Conv2D(32, kernel_size=3, activation='relu'))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))
        self.model.add(Flatten())
        self.model.add(Dense(10, activation='softmax'))
        self.model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])    

    def train(self, totTrain=100_000, epochs=10, ratio=(1,1)): #Real to Fake
        sum = ratio[0] + ratio[1]
        realTrain = ( totTrain // sum ) * ratio[0] 
        fakeTrain = ( totTrain // sum ) * ratio[1] 

        X_train_raw, X_val, y_train_raw, y_val = train_test_split(train_data, train_label, test_size=0.33, random_state=43)

        X_train = np.concatenate((X_train_raw[:realTrain], X_fake[:fakeTrain]))
        y_train = np.concatenate((y_train_raw[:realTrain], y_fake[:fakeTrain]))
        
        print(X_train.shape)
        print(X_train_raw[:realTrain].shape)
        print(X_fake[:fakeTrain].shape)

        # model.summary()

        self.model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs)
        os.makedirs('classif_history/', exist_ok=True)

        root = 'classif_history/'
        with open(root + 'classif_'+ str(totTrain) + '_' + str(ratio[0]) + '_' + str(ratio[1]) + '.csv', 'w') as f:
            for key in self.model.history.history.keys():
                f.write("%s,%s\n"%(key,self.model.history.history[key]))

        X_test_reshaped = X_test.reshape(10000, 28, 28, 1)
        loss, acc = self.model.evaluate(X_test_reshaped, to_categorical(y_test))
        print(acc)
        with open(root + 'classif_acc.csv', 'a') as b:
            b.write(str(totTrain) + ',' + str(ratio[0]) + ',' + str(ratio[1]) + ',' + "%f\n"%(acc))
            

In [ ]:
ratio_array = [ (0,1), (1,0), (1,1), (2,1), (1,2), (3,1), (1,3), (5,1), (1,5), (10,1), (1,10)]
train_array = [ 10000 ]




for num in train_array:
    for ratio in ratio_array:
        
#         del cl
        if (ratio != ratio_array[0]) & (num != train_array[0]):
            del cl
        cl = classifier()


#         cl.reset_weights()
        cl.train(totTrain=num, epochs=10, ratio=ratio)


Instructions for updating:
Colocations handled automatically by placer.
(10000, 28, 28, 1)
(0, 28, 28, 1)
(10000, 28, 28, 1)
Instructions for updating:
Use tf.cast instead.
Train on 10000 samples, validate on 19800 samples
Epoch 1/10
10000/10000 [==============================] - 6s 576us/step - loss: 12.9970 - acc: 0.1908 - val_loss: 12.6971 - val_acc: 0.2111
Epoch 2/10
10000/10000 [==============================] - 3s 255us/step - loss: 10.8552 - acc: 0.3218 - val_loss: 9.7197 - val_acc: 0.3930
Epoch 3/10
10000/10000 [==============================] - 3s 253us/step - loss: 8.5639 - acc: 0.4623 - val_loss: 8.1834 - val_acc: 0.4871
Epoch 4/10
10000/10000 [==============================] - 3s 251us/step - loss: 7.6946 - acc: 0.5156 - val_loss: 6.7666 - val_acc: 0.5716
Epoch 5/10
10000/10000 [==============================] - 3s 252us/step - loss: 6.5096 - acc: 0.5891 - val_loss: 6.6869 - val_acc: 0.5768
Epoch 6/10
10000/10000 [==============================] - 3s 252us/step - loss: 6.40

10000/10000 [==============================] - 1s 61us/step
0.8506
(10000, 28, 28, 1)
(7500, 28, 28, 1)
(2500, 28, 28, 1)
Train on 10000 samples, validate on 19800 samples
Epoch 1/10
10000/10000 [==============================] - 3s 303us/step - loss: 13.1471 - acc: 0.1786 - val_loss: 13.1026 - val_acc: 0.1819
Epoch 2/10
10000/10000 [==============================] - 3s 265us/step - loss: 11.9148 - acc: 0.2561 - val_loss: 11.3192 - val_acc: 0.2944
Epoch 3/10
10000/10000 [==============================] - 3s 265us/step - loss: 11.2130 - acc: 0.3023 - val_loss: 11.3305 - val_acc: 0.2939
Epoch 4/10
10000/10000 [==============================] - 3s 266us/step - loss: 10.0382 - acc: 0.3711 - val_loss: 9.7133 - val_acc: 0.3940
Epoch 5/10
10000/10000 [==============================] - 3s 266us/step - loss: 7.0423 - acc: 0.5428 - val_loss: 5.0822 - val_acc: 0.6652
Epoch 6/10
10000/10000 [==============================] - 3s 268us/step - loss: 1.3248 - acc: 0.8301 - val_loss: 0.2462 - val_acc: 

Epoch 3/10
9999/9999 [==============================] - 3s 305us/step - loss: 4.9554 - acc: 0.6783 - val_loss: 3.7465 - val_acc: 0.7422
Epoch 4/10
9999/9999 [==============================] - 3s 296us/step - loss: 3.4558 - acc: 0.7713 - val_loss: 3.6572 - val_acc: 0.7511
Epoch 5/10
9999/9999 [==============================] - 3s 284us/step - loss: 3.3630 - acc: 0.7805 - val_loss: 3.7982 - val_acc: 0.7340
Epoch 6/10
9999/9999 [==============================] - 3s 287us/step - loss: 3.3411 - acc: 0.7828 - val_loss: 3.5934 - val_acc: 0.7507
Epoch 7/10
9999/9999 [==============================] - 3s 290us/step - loss: 3.3087 - acc: 0.7896 - val_loss: 3.6201 - val_acc: 0.7507
Epoch 8/10
9999/9999 [==============================] - 3s 308us/step - loss: 3.3026 - acc: 0.7911 - val_loss: 3.6116 - val_acc: 0.7504
Epoch 9/10
9999/9999 [==============================] - 3s 295us/step - loss: 3.3192 - acc: 0.7858 - val_loss: 3.6355 - val_acc: 0.7506
Epoch 10/10
10000/10000 [=======================

In [ ]:
ratio_array = [ (1,2) ]
train_array = [ 150_000 ]



for num in train_array:
    for ratio in ratio_array:
        del cl
#         if (ratio != ratio_array[0]) & (num != train_array[0]):
#             del cl
        cl = classifier()
        cl.train(totTrain=num, epochs=25, ratio=ratio)


(140200, 28, 28, 1)
(40200, 28, 28, 1)
(100000, 28, 28, 1)
Train on 140200 samples, validate on 19800 samples
Epoch 1/25
140200/140200 [==============================] - 25s 177us/step - loss: 2.9067 - acc: 0.7931 - val_loss: 0.1743 - val_acc: 0.9516
Epoch 2/25
140200/140200 [==============================] - 24s 171us/step - loss: 0.0871 - acc: 0.9735 - val_loss: 0.1175 - val_acc: 0.9673
Epoch 3/25
140200/140200 [==============================] - 24s 171us/step - loss: 0.0706 - acc: 0.9784 - val_loss: 0.1089 - val_acc: 0.9687
Epoch 4/25
140200/140200 [==============================] - 24s 173us/step - loss: 0.0614 - acc: 0.9814 - val_loss: 0.1232 - val_acc: 0.9685
Epoch 5/25
140200/140200 [==============================] - 24s 170us/step - loss: 0.0603 - acc: 0.9819 - val_loss: 0.1361 - val_acc: 0.9660
Epoch 6/25
 70240/140200 [==============>...............] - ETA: 11s - loss: 0.0571 - acc: 0.9829